In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cpu


In [11]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))
#print(text[:200])

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

232328
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [12]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])
print(encode)
print(decode)


encode_hello = encode('hello')
decode_hello = decode(encode_hello)

print(encode_hello)
print(decode_hello)


<function <lambda> at 0x1407d71a0>
<function <lambda> at 0x1407d6660>
[61, 58, 65, 65, 68]
hello


In [13]:
import torch

with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))


string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

#encode = lambda s : [string_to_int[c] for c in s]
#decode = lambda l: ''.join([int_to_string[i] for i in l])


data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

232328
tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [22]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    X, y = X.to(device), y.to(device) #This will change to gpu if available. Define device first
    return X, y

X, y = get_batch('train')
print('input:')
print(X)
print('target:')
print(y)

input:
tensor([[69, 58, 68, 69, 65, 58,  1, 76],
        [58,  1, 60, 62, 71, 65, 11,  0],
        [67,  0, 76, 54, 72,  1, 55, 71],
        [61, 58, 66, 72, 58, 65, 75, 58]])
target:
tensor([[58, 68, 69, 65, 58,  1, 76, 62],
        [ 1, 60, 62, 71, 65, 11,  0,  0],
        [ 0, 76, 54, 72,  1, 55, 71, 58],
        [58, 66, 72, 58, 65, 75, 58, 72]])


In [23]:
X = train_data[:block_size]
y = val_data[1:block_size+1]

for t in range(block_size):
    context = X[:t+1]
    target = y[t]
    print('When input is', context, 'target is ', target)

When input is tensor([80]) target is  tensor(76)
When input is tensor([80,  1]) target is  tensor(67)
When input is tensor([80,  1,  1]) target is  tensor(1)
When input is tensor([80,  1,  1, 28]) target is  tensor(60)
When input is tensor([80,  1,  1, 28, 39]) target is  tensor(71)
When input is tensor([80,  1,  1, 28, 39, 42]) target is  tensor(54)
When input is tensor([80,  1,  1, 28, 39, 42, 39]) target is  tensor(75)
When input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is  tensor(58)


In [24]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [25]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


n.5haIta,YneW1Yrye!-&0.W2U tb41;EF-3'1rSmVOe:H99naBdk-TiT&L,M":g﻿cuorU 2OH5GrbIrgr3c-H.F'rQJ)f;2H[Mai,m
yyr?&U2cuLF8RXw?K,)9(jCR?xmR?YLFAXC7uA?K9Cuo﻿u
V28nrY];aoSyCK9M,Y-RGb:sqy-2wLl[]K,9[ScuQU7.xE_yg,j*9pPYssHlhwtyDJHK?vj3cymyPU2,Ak6BYkb8?
VSAlMhCkdNYG23&i﻿WBr,YcgRP.-"tU(mPv,YF!:0Q&K9NG7wT6z1OfWq1OQ8!Nkr3bR.t4um8tr2Sm[GF6BYet_ljzxq"mcg;kNY.1tl:N0o kpko.ZN*76FXx[Vx]KPB
8yP1ta,)TL&VNZ*Z**F49ya﻿﻿Ki﻿ZpZzO7" 9RJYOe;﻿﻿Eb?F8.F&QGHu1RTlGopEmP5zjgf5;PQ]KyH'H9M
iu]KBfHTLTOeS)cs5HM'AuV)Yd.l:1Etx!e tXlik]m


In [27]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 3.160, val loss: 3.171
step: 250, train loss: 3.101, val loss: 3.143
step: 500, train loss: 3.090, val loss: 3.115
step: 750, train loss: 3.088, val loss: 3.108
step: 1000, train loss: 3.058, val loss: 3.101
step: 1250, train loss: 3.029, val loss: 3.043
step: 1500, train loss: 3.010, val loss: 3.027
step: 1750, train loss: 2.985, val loss: 3.026
step: 2000, train loss: 2.974, val loss: 2.996
step: 2250, train loss: 2.947, val loss: 2.990
step: 2500, train loss: 2.931, val loss: 2.991
step: 2750, train loss: 2.888, val loss: 2.954
step: 3000, train loss: 2.937, val loss: 2.939
step: 3250, train loss: 2.911, val loss: 2.929
step: 3500, train loss: 2.871, val loss: 2.914
step: 3750, train loss: 2.886, val loss: 2.881
step: 4000, train loss: 2.868, val loss: 2.882
step: 4250, train loss: 2.837, val loss: 2.873
step: 4500, train loss: 2.849, val loss: 2.856
step: 4750, train loss: 2.793, val loss: 2.838
step: 5000, train loss: 2.810, val loss: 2.815
step: 5250, train l

***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***\n,
    \n,
    1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.\n,
    
    2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function\n,
    
    3. **Momentum**: Momentum is an extension of SGD that adds a \"momentum\" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.\n,
    
    4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.\n,
    
    5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.\n,
    
    6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.\n,
    \n,
    find more optimizers and details at torch.optim

In [82]:
nums = [1, 2, 3, 4, 5]
total = 0

for num in nums:
    if num % 2 == 0:
        total += num
    else:
        total -= num

print(total)

-3
